# Functional Programming 
## What, Why and How

We will try to explain the first principles of functional programming by trying as much as possible to avoid big FP words.

We will use python for this exercise, because it's easy to read; and it provides all the functional features required for understanding concepts described in this document.

# A simple problem

## Count the number of words in a large file

You have a file which may have millions of lines. we need to count the number of words on the file.

### A simple solution

In [ ]:
def count_w(filename):
    count = 0
    offset = 0
    file = open(filenames)
    for line in file:
        for w in line.split():
            count += 1
    return count
# But This Doesnt Scale!

When things don't scale,
![title](img/parallel.jpg)

Lets try the same example with Threads

In [ ]:
count = 0

def count_w_chunk(file_chunk):
    global count
    count = 0
    offset = 0
    file = open(filenames)
    for line in file:
        for w in line.split():
            count += 1

#actual driver code
def count_w(file):
    threads = [] # empty thread list
    chunks = file.make_chunks()
    # magic function parallelize
    for chunk in chunks:
        parallelize(count_w_chunks(chunk))
    print (count)


# But the code doesn't work!
## Race Conditions

Lets take a simpler example

In [ ]:
count = 0
def increment(delta):
    global count
    temp = count
    temp += delta
    count = temp
    return count

# what will happen if increment(1) is run parallelly 1000 times ?

The result will be less than 1000 most of the time.
We will need some sort of locking/synchronization mechanism to avoid writing the same value to the variable over and over again.

In [ ]:
# this will work, but sucks
count = 0
def increment(delta):
    global count
    with synchronized(count):
        count += delta
    return count


The culprit in the above code is the assignment. Re-assignment to be precise. Once initialized, a `variable` can have different values at different times, depending on what we are `assigning` to it.
If we had code that did not change the `state` of `count` variable, we would'nt need to synchronize the assignments.

# Principle 1
## Avoid shared state change wherever possible

# Functions with Side Effects

When a function changes it `state` while it's being executed, and the `state` change is persisted between execution.
Put simply but inaccurately, if a function might different values while invoking it with the same parameters

#### Note
Functions that changes `state` outside it's scope also cause side effects. For example,
```code
def foo():
    with open('filename', 'a') as f:
        f.write('side effect')
    return 0
```
The above function returns 0 all the time, but still changes the `state` of the file system

# What is a function?
### We have functions in Mathematics 

```math
f(x) = x^2 + 2x + 3
```

They do no state manipulation. f(25) is 678 no matter how many times you call it.
These kind of functions are called `pure` functions.

Read more about pure functions [here](https://en.wikipedia.org/wiki/Pure_function)

While you are at it, read about [referential transparency](https://en.wikipedia.org/wiki/Referential_transparency)

Its okay if you don't get it.


In [ ]:
# Modeling a 'pure' function
def f(x):
    return x*x + 2*x + 3

In [ ]:
f(9)

# Say NO to Mutations

# Imagine a world without assignments

* No mutations no race conditions
* Free parallelism
* Modular code
* Mathematical reasonability
* Your IDE can tell you if your code is right

# some fun with fun-ctions
#### I do not apologize for my puns

In [ ]:
# frange(0,1,0.1) == [0, .1, .2, .3, .4 ... .9]
def frange(a,b,delta):
    i = a
    while i < b:
        yield i
        i += delta
        
# yes there are assignments in the above code, they are avoidable using recursion

In [ ]:
# integration; approximation of area under the curve. for more accurate value, reduce the value of err
def integ(f, a,b):
    err = 0.01
    return sum( [f(x) * err for x in frange(a,b,err)]
    )


In [ ]:
def square(x):
    return x*x

integ(square, 0, 2)

In [ ]:
# functions returning functions

def add_value(value):
    def adder(x):
        return x + value
    return adder

In [ ]:
add5 = add_value(5)

In [ ]:
add5(22)

  ### A note about Lambda
  
  lambda lets you write functions similar to mathematical expressions. It saves mind-space in remembering definitions, and sometimes makes statements more readable.
  
As a rule of thumb, use lambdas if the return value is a simple expression

In [ ]:
def square(x):
    return x*x

# is the same as
lambda x: x*x


In [ ]:
# functions taking functions as argument

def fogger(f,g):
    def fog(x):
        return f(g(x))
    return fog

In [ ]:
def add6(x):
    return x + 6

new_function = fogger (add6, add5)
new_function(20)

# Map Reduce & Filter